In [30]:
import pandas as pd
import numpy as np
import pickle
from faker import Faker

# Initialize Faker
fake = Faker()

admission_types = [
    'URGENT', 'ELECTIVE', 'EW EMER.', 'DIRECT EMER.', 'EU OBSERVATION',
    'OBSERVATION ADMIT', 'DIRECT OBSERVATION',
    'AMBULATORY OBSERVATION', 'SURGICAL SAME DAY ADMISSION'
]
admission_locations = [
    'TRANSFER FROM HOSPITAL', 'TRANSFER FROM SKILLED NURSING FACILITY',
    'INTERNAL TRANSFER TO OR FROM PSYCH', 'PHYSICIAN REFERRAL',
    'EMERGENCY ROOM', 'PACU', 'PROCEDURE SITE',
    'WALK-IN/SELF REFERRAL', 'INFORMATION NOT AVAILABLE',
    'CLINIC REFERRAL'
]
insurance_types = [
    'Medicaid', 'Medicare', 'Other'
]
languages = [
    'ENGLISH', '?'
]
marital_statuses = [
    'SINGLE', 'MARRIED', 'DIVORCED', 'WIDOWED'
]
races = [
    'BLACK/CAPE VERDEAN', 'HISPANIC/LATINO - PUERTO RICAN', 'WHITE',
    'UNKNOWN', 'OTHER', 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - SALVADORAN', 'UNABLE TO OBTAIN',
    'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'HISPANIC/LATINO - CUBAN',
    'PATIENT DECLINED TO ANSWER', 'WHITE - BRAZILIAN',
    'HISPANIC OR LATINO'
]

drugs = [
    'Midodrine', 'Multivitamins W/minerals', 'Sodium Chloride',
    'Nephrocaps', 'Nicotine Polacrilex', 'OLANZapine', 'Ondansetron',
    'OxyCODONE (Immediate Release)'
]

diagnoses_types = [
    'Urinary tract infection, site not specified',
    'Acute respiratory failure',
    'Asthma, unspecified type, unspecified',
    'Solitary pulmonary nodule',
    'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
    'Dysphagia, oropharyngeal phase', 'Esophageal reflux', 'Hypoxemia',
    'Methicillin resistant pneumonia due to Staphylococcus aureus',
    'Attention deficit disorder with hyperactivity'
]

def generate_patients(n):
    patients = []
    for i in range(n):
        gender = np.random.choice(['M', 'F'])
        patients.append({
            'patient_id': i,
            'name': fake.name_male() if gender == "M" else fake.name_female(),
            'age': np.random.randint(1, 90),
            'gender': gender,
            'insurance': np.random.choice(insurance_types),
            'language': np.random.choice(languages),
            'maritalStatus': np.random.choice(marital_statuses),
            'race': np.random.choice(races),
        })
    return pd.DataFrame(patients)

def generate_staff(n):
    staff = []
    for i in range(n):
        staff.append({
            'staff_id': i,
            'name': fake.name(),
            'role': np.random.choice(['Physician', 'Nurse', 'Admin']),
            'shift_start': fake.time(),
            'shift_end': fake.time()
        })
    return pd.DataFrame(staff)

def generate_admissions(patients:pd.DataFrame):
    admissions = []
    for i in range(len(patients)):
        admissions.append({
            'adm_id': i,
            'patient_id': patients.iloc[i].patient_id,
            'admit_time': fake.date_time(),
            'discharge_time': None,
            'type': np.random.choice(admission_types),
            'location': np.random.choice(admission_locations),
            
        })
    return pd.DataFrame(admissions)

def generate_omr(admissions):
    table = []
    for i in range(len(admissions)):
        omr = {
            'omr_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'weight': np.random.randint(60, 120),
            ##'height': np.random.randint(150, 220)/ 10.0, #not used in modeling
            'bp_systolic': np.random.randint(80, 220),
            'bp_diastolic': np.random.randint(60, 120),
        }
        table.append(omr)
    return pd.DataFrame(table)

def generate_prescriptions(admissions):
    prescriptions = []
    for i in range(len(admissions)):
        prescriptions.append({
            'prescription_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'drug': np.random.choice(drugs),
        })
    return pd.DataFrame(prescriptions)

def generate_diagnoses(admissions:pd.DataFrame):
    diag = []
    for i in range(len(admissions)):
        diag.append({
            'diagnosis_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'diagnosis': np.random.choice(diagnoses_types),
        })
    return pd.DataFrame(diag)

def generate_rooms(n):
    rooms = []
    beds = []
    bed_taken = 0
    room_id = 0
    bed_id = 0
    for i in range(n):
        if i % 2 == 0:#create new room
            rooms.append({
                'room_id': room_id,
                'capacity': 1,
            })
            beds.append({
                "bed_id": bed_id,
                "room": room_id,
                "adm_id": None
            })
            room_id += 1
        else:
            room = np.random.choice(rooms)#add bed to existing room
            room["capacity"] += 1
            beds.append({
                "bed_id": bed_id,
                "room": room["room_id"],
                "adm_id": None
            })
        bed_id += 1

    return pd.DataFrame(rooms), pd.DataFrame(beds)

def assign_beds(admissions, beds):
    for i in range(len(admissions)):
        adm = admissions.iloc[i].adm_id
        bed = beds[beds.adm_id.isna()].sample(1).iloc[0,0]  #choose random empty bed
        beds.loc[bed, "adm_id"] = adm.astype(int)
        print(bed)

In [31]:
staff = generate_staff(10)
patients = generate_patients(10)
admissions = generate_admissions(patients)
admissions.discharge_time = pd.to_datetime(admissions.discharge_time)
omr = generate_omr(admissions)
prescriptions = generate_prescriptions(admissions)
diagnoses = generate_diagnoses(admissions)
rooms, beds = generate_rooms(20)
assign_beds(admissions, beds)

17
19
0
12
3
13
15
10
16
1


In [32]:
staff.head()

,staff_id,name,role,shift_start,shift_end
0,0,Jaclyn Fernandez,Nurse,22:58:29,19:57:49
1,1,Jonathan Livingston II,Nurse,07:37:09,18:07:00
2,2,Maria Baird MD,Physician,22:03:36,13:32:43
3,3,Michelle Lewis,Physician,00:11:06,00:32:51
4,4,Tyler Alexander,Physician,00:25:55,17:16:22


In [33]:
patients.head()

,patient_id,name,age,gender,insurance,language,maritalStatus,race
0,0,Michael Clay,3,M,Medicare,ENGLISH,SINGLE,UNKNOWN
1,1,Joshua Riley,30,M,Other,?,DIVORCED,WHITE
2,2,Zachary Kim,78,M,Medicaid,ENGLISH,MARRIED,WHITE - BRAZILIAN
3,3,Steven Vasquez,44,M,Other,?,SINGLE,HISPANIC/LATINO - PUERTO RICAN
4,4,Allen David,12,M,Medicare,?,DIVORCED,PORTUGUESE


In [34]:
admissions

,adm_id,patient_id,admit_time,discharge_time,type,location
0,0,0,1989-11-28 01:53:33.902606,NaT,DIRECT EMER.,INFORMATION NOT AVAILABLE
1,1,1,2006-11-01 23:46:18.923388,NaT,DIRECT EMER.,WALK-IN/SELF REFERRAL
2,2,2,2012-03-19 19:06:02.419464,NaT,AMBULATORY OBSERVATION,PROCEDURE SITE
3,3,3,2014-03-29 16:06:49.847335,NaT,AMBULATORY OBSERVATION,PHYSICIAN REFERRAL
4,4,4,1997-01-24 16:03:55.795988,NaT,URGENT,PACU
5,5,5,1987-04-01 12:16:45.562816,NaT,EU OBSERVATION,EMERGENCY ROOM
6,6,6,2024-07-15 16:53:22.755505,NaT,OBSERVATION ADMIT,CLINIC REFERRAL
7,7,7,1981-01-03 12:49:34.672635,NaT,URGENT,PROCEDURE SITE
8,8,8,1980-09-13 00:19:47.169325,NaT,DIRECT OBSERVATION,CLINIC REFERRAL
9,9,9,2013-05-11 12:50:40.370972,NaT,DIRECT EMER.,WALK-IN/SELF REFERRAL


In [35]:
admissions.iloc[0,0]

0

In [36]:
omr.head()

,omr_id,adm_id,weight,bp_systolic,bp_diastolic
0,0,0,112,164,102
1,1,1,118,137,65
2,2,2,76,85,97
3,3,3,69,192,84
4,4,4,118,153,64


In [37]:
prescriptions.head()

,prescription_id,adm_id,drug
0,0,0,Nicotine Polacrilex
1,1,1,Sodium Chloride
2,2,2,OxyCODONE (Immediate Release)
3,3,3,OxyCODONE (Immediate Release)
4,4,4,OxyCODONE (Immediate Release)


In [38]:
diagnoses.head()

,diagnosis_id,adm_id,diagnosis
0,0,0,Streptococcus infection in conditions classifi...
1,1,1,"Asthma, unspecified type, unspecified"
2,2,2,"Asthma, unspecified type, unspecified"
3,3,3,"Urinary tract infection, site not specified"
4,4,4,Streptococcus infection in conditions classifi...


In [39]:
rooms

,room_id,capacity
0,0,3
1,1,3
2,2,3
3,3,3
4,4,2
5,5,1
6,6,2
7,7,1
8,8,1
9,9,1


In [40]:
beds

,bed_id,room,adm_id
0,0,0,2
1,1,0,9
2,2,1,None
3,3,1,4
4,4,2,None
5,5,2,None
6,6,3,None
7,7,3,None
8,8,4,None
9,9,0,None


In [41]:
with open('app/data/staff.csv', 'w') as f:
    staff.to_csv(f, index=False)
with open('app/data/patients.csv', 'w') as f:
    patients.to_csv(f, index=False)
with open('app/data/admissions.csv', 'w') as f:
    admissions.to_csv(f, index=False)
with open('app/data/omr.csv', 'w') as f:
    omr.to_csv('app/data/omr.csv', index=False)
with open('app/data/prescriptions.csv', 'w') as f:
    prescriptions.to_csv("app/data/prescriptions.csv", index=False)
with open("app/data/diagnoses.csv", "w") as f:
    diagnoses.to_csv("app/data/diagnoses.csv", index=False)
with open("app/data/rooms.csv", "wb") as f:
    rooms.to_csv(f, index=False)
with open("app/data/beds.csv", "wb") as f:
    beds.to_csv(f, index=False)

In [42]:
with open("app/data/admission_types.pkl", "wb") as f:
    pickle.dump(admission_types, f)
with open("app/data/admission_locations.pkl", "wb") as f:
    pickle.dump(admission_locations, f)
with open("app/data/insurance_types.pkl", "wb") as f:
    pickle.dump(insurance_types, f)
with open("app/data/languages.pkl", "wb") as f:
    pickle.dump(languages, f)
with open("app/data/marital_statuses.pkl", "wb") as f:
    pickle.dump(marital_statuses, f)
with open("app/data/races.pkl", "wb") as f:
    pickle.dump(races, f)
with open("app/data/drugs.pkl", "wb") as f:
    pickle.dump(drugs, f)
with open("app/data/diagnoses_types.pkl", "wb") as f:
    pickle.dump(diagnoses_types, f)